Installing required libraries for stepfunctions

In [1]:
#  import sys
#  !{sys.executable} -m pip install --upgrade pip
#  !{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
#  !{sys.executable} -m pip install -qU "stepfunctions>=2.0.0"
#  !{sys.executable} -m pip show sagemaker stepfunctions

# 1. Setting up notebook with parameters and libraries

In [1]:
# importing Python libraies
import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from sagemaker.processing import Processor,ProcessingInput, ProcessingOutput
import calendar
import time
import sagemaker
from sagemaker.inputs import TrainingInput
import boto3
from sagemaker.network import NetworkConfig

stepfunctions.set_stream_logger(level=logging.INFO)

import stepfunctions
import logging



### Defining paramters

This needs to be changed if we are taking it to different environment

In [2]:
v_workflow_execution_role = "arn:aws:iam::014257795134:role/ds-mlops-stepfunction-role" # Step function IAM role ARN
v_preprocessing_iam_role = "arn:aws:iam::014257795134:role/ds-mlops-sagemaker-role" # IAM role for preprocessing container
v_preprocessing_instance_type = "ml.m5.xlarge" # Instance type for preprocessing container it changes as per workload
v_s3_input_bucket = "ds-mlops-s3" # S3 bucket for input and output data
v_prefix_for_input_data = "data/input/train_baseline.csv"  # Prefix where data is stored
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-01d629a900f9b4d92"]
subnets = ["subnet-07bd1dfe6aee76227",
           "subnet-076950ecc89d4340b",
           "subnet-0c5a462cb45a14bab"]
config_bucket = "ds-mlops-s3"


## 2. Defining preprocessing jobs

In [4]:
# Defining environment config for baseline jobs
environment = {
                "dataset_format": "{\"csv\":{\"header\": true,\"output_columns_position\": \"START\"}}",
                "dataset_source": "/opt/ml/processing/input/baseline_dataset_input",
                "output_path": "/opt/ml/processing/output",
                "publish_cloudwatch_metrics": "Disabled"
        }


In [5]:
# Here we are creating baseline preprocesor
baseline_processor = Processor(image_uri='156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer',
                     role=v_preprocessing_iam_role,
                     instance_count=1,
                     instance_type=v_preprocessing_instance_type,
                     network_config = NetworkConfig(security_group_ids = sec_groups, subnets = subnets),
                     env=environment)

In [6]:
input_data = "s3://{}/{}".format(v_s3_input_bucket,v_prefix_for_input_data)
inputs = [
    ProcessingInput(
        source=input_data, destination="/opt/ml/processing/input/baseline_dataset_input", input_name="input_data"
    )
]

outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output",
        destination="s3://{}/{}".format(config_bucket,"customonitor/"),
        output_name="tr_data",
    )
]
gmt = time.gmtime()
ts = calendar.timegm(gmt)
baseline_name = "baseline-{}".format(ts)



In [7]:
print("s3://{}/{}".format(config_bucket,"customonitor"))


s3://ds-mlops-s3/customonitor


In [8]:
print(input_data)

s3://ds-mlops-s3/data/input/train_baseline.csv


In [9]:
baseline_preprocessing_step = steps.ProcessingStep(
    state_id='Baseline', 
    processor=baseline_processor,
    job_name=baseline_name, 
    inputs=inputs, 
    outputs=outputs, 
    experiment_config=None, 
    wait_for_completion=True
)

## 3. Step Function

In [10]:
# First we chain the start pass state,preprocessing_step,
basic_path=Chain([baseline_preprocessing_step])

In [11]:
# Next, we define the workflow
basic_workflow = Workflow(
    name="ds-mlops-baseline-job-v12",
    definition=basic_path,
    role=v_workflow_execution_role
)

#Render the workflow
basic_workflow.render_graph()

## 3.1 Create the workflow on AWS Step Functions

Create the workflow in AWS Step Functions with [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create).

In [12]:
basic_workflow.create()

[ERROR] A workflow with the same name already exists on AWS Step Functions. To update a workflow, use Workflow.update().


'arn:aws:states:us-east-1:014257795134:stateMachine:ds-mlops-baseline-job-v12'

In [13]:
basic_workflow.update(definition=basic_workflow.definition,role=basic_workflow.role)

[INFO] Workflow updated successfully on AWS Step Functions. All execute() calls will use the updated definition and role within a few seconds. 


'arn:aws:states:us-east-1:014257795134:stateMachine:ds-mlops-baseline-job-v12'

In [14]:
basic_workflow_execution = basic_workflow.execute(
    inputs={
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


## 3.2 Review the execution progress

Render workflow progress with the [render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress).

This generates a snapshot of the current state of your workflow as it executes. This is a static image. Run the cell again to check progress. 

In [15]:
basic_workflow_execution.render_progress()

## 4 Downloading generated report on notebook

In [16]:
# Downloading contraint file for evaluation
!aws s3 cp s3://$config_bucket/monitoring/constraints.json .

download: s3://ds-mlops-s3/monitoring/constraints.json to ./constraints.json


In [17]:
# Dowloading stats file for evaluation
!aws s3 cp s3://$config_bucket/monitoring/statistics.json .

download: s3://ds-mlops-s3/monitoring/statistics.json to ./statistics.json


We can perform analysis on this file and put it on git hub

**Note :**
In order to refer it for modified location in monitoring schedule lambda function we need to changes the baseline stats location to prefix custom_monitoring/ from monitoring


In [18]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
input_s3_path='s3://ds-mlops-s3/data/input/train_baseline.csv'
output_s3_path='s3://ds-mlops-s3/monitoring/'
my_default_monitor = DefaultModelMonitor(
role=v_preprocessing_iam_role,
instance_count=1,
instance_type="ml.m5.xlarge",
volume_size_in_gb=20,
max_runtime_in_seconds=3600,
)



my_default_monitor.suggest_baseline(
baseline_dataset=input_s3_path,#'s3://sagemaker-us-east-1-525102048888/sagemaker/DEMO-ModelMonitor/baselining/data/training-dataset-with-header.csv',
dataset_format=DatasetFormat.csv(header=True),
output_s3_uri=output_s3_path,#'s3://wipcoe-datalake-init-s3-mlops-config/customonitor',
wait=True,
)


Job Name:  baseline-suggestion-job-2022-02-17-22-08-35-389
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ds-mlops-s3/data/input/train_baseline.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ds-mlops-s3/monitoring/', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
............................2022-02-17 22:13:03,534 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-02-17 22:13:04.096953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-17 22:13:04.096983: I tensorflow/st

2022-02-17 22:13:14,869 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start resourcemanager, return code 1
2022-02-17 22:13:14,869 - bootstrap - INFO - Running command: /usr/hadoop-3.0.0/bin/yarn --daemon start nodemanager
2022-02-17 22:13:16,980 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start nodemanager, return code 1
2022-02-17 22:13:16,980 - bootstrap - INFO - Running command: /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver
2022-02-17 22:13:19,072 - bootstrap - INFO - Failed to run /usr/hadoop-3.0.0/bin/yarn --daemon start proxyserver, return code 1
2022-02-17 22:13:19,073 - DefaultDataAnalyzer - INFO - Total number of hosts in the cluster: 1
2022-02-17 22:13:29,082 - DefaultDataAnalyzer - INFO - Running command: bin/spark-submit --master yarn --deploy-mode client --conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider --conf spark.serializer=org.apache.spark.serializer.Kry

2022-02-17 22:13:45 INFO  YarnSchedulerBackend$YarnDriverEndpoint:54 - Registered executor NettyRpcEndpointRef(spark-client://Executor) (10.0.165.60:48568) with ID 1
2022-02-17 22:13:46 INFO  BlockManagerMasterEndpoint:54 - Registering block manager algo-1:41005 with 5.8 GB RAM, BlockManagerId(1, algo-1, 41005, None)
2022-02-17 22:14:01 INFO  YarnClientSchedulerBackend:54 - SchedulerBackend is ready for scheduling beginning after waiting maxRegisteredResourcesWaitingTime: 30000(ms)
2022-02-17 22:14:01 WARN  SparkContext:66 - Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp' appears to be on the local filesystem.
2022-02-17 22:14:01 INFO  DatasetReader:132 - Files to process:List(file:///opt/ml/processing/input/baseline_dataset_input/train_baseline.csv)
2022-02-17 22:14:01 INFO  SharedState:54 - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/usr/spark-2.3.1/spark-war

2022-02-17 22:14:11 INFO  TaskSetManager:54 - Finished task 2.0 in stage 2.0 (TID 4) in 2509 ms on algo-1 (executor 1) (1/3)
2022-02-17 22:14:11 INFO  BlockManagerInfo:54 - Added rdd_11_1 in memory on algo-1:41005 (size: 4.9 MB, free: 5.8 GB)
2022-02-17 22:14:11 INFO  TaskSetManager:54 - Finished task 0.0 in stage 2.0 (TID 2) in 2929 ms on algo-1 (executor 1) (2/3)
2022-02-17 22:14:12 INFO  TaskSetManager:54 - Finished task 1.0 in stage 2.0 (TID 3) in 3419 ms on algo-1 (executor 1) (3/3)
2022-02-17 22:14:12 INFO  DAGScheduler:54 - ShuffleMapStage 2 (collect at AnalysisRunner.scala:313) finished in 3.440 s
2022-02-17 22:14:12 INFO  YarnScheduler:54 - Removed TaskSet 2.0, whose tasks have all completed, from pool 
2022-02-17 22:14:12 INFO  DAGScheduler:54 - looking for newly runnable stages
2022-02-17 22:14:12 INFO  DAGScheduler:54 - running: Set()
2022-02-17 22:14:12 INFO  DAGScheduler:54 - waiting: Set(ResultStage 3)
2022-02-17 22:14:12 INFO  DAGScheduler:54 - failed: Set()
2022-02-17 

2022-02-17 22:14:20 INFO  MemoryStore:54 - Block broadcast_14 stored as values in memory (estimated size 31.0 KB, free 1457.7 MB)
2022-02-17 22:14:20 INFO  MemoryStore:54 - Block broadcast_14_piece0 stored as bytes in memory (estimated size 12.9 KB, free 1457.7 MB)
2022-02-17 22:14:20 INFO  BlockManagerInfo:54 - Added broadcast_14_piece0 in memory on 10.0.165.60:33273 (size: 12.9 KB, free: 1458.4 MB)
2022-02-17 22:14:20 INFO  SparkContext:54 - Created broadcast 14 from broadcast at DAGScheduler.scala:1039
2022-02-17 22:14:20 INFO  DAGScheduler:54 - Submitting 3 missing tasks from ResultStage 11 (MapPartitionsRDD[67] at treeReduce at KLLRunner.scala:107) (first 15 tasks are for partitions Vector(0, 1, 2))
2022-02-17 22:14:20 INFO  YarnScheduler:54 - Adding task set 11.0 with 3 tasks
2022-02-17 22:14:20 INFO  TaskSetManager:54 - Starting task 0.0 in stage 11.0 (TID 23, algo-1, executor 1, partition 0, PROCESS_LOCAL, 8349 bytes)
2022-02-17 22:14:20 INFO  TaskSetManager:54 - Starting task 

## Clean-up steps

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

In [6]:
# # Clean up end point
# client = boto3.client("sagemaker", region_name=v_region)
# response=client.delete_endpoint(EndpointName=endpoint_name)